### Contents  
**_I. Data preprocessing_**
> 1. Read dataset  
> 2. Missing values handling
>> a) `train.csv`  
>> b) `test.csv`
> 3. Data manipulation
>> a) `Date`  
>> b) `IsHoliday`  
>> c) `Store`  
>> d) `Promotion1`, ... , `Promotion5`
> 4. Add new features
> 5. Remove not using features  

**_II. Modeling_**  
> 1. Divide `train.csv` into training data and predicting data
> 2. Choose a suitable model
>> a) XGBooster  
>> b) Random Forest  

**_III. Submission_**

----
# I. Data preprocessing

## 1. Read dataset

In [1]:
import pandas as pd

train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
submission = pd.read_csv("dataset/sample_submission.csv")

## 2. Missing values handling

### a) `train.csv`

In [2]:
train = train.fillna(0)

### b) `test.csv`

In [3]:
means = {
    "Promotion1":test["Promotion1"].mean(), "Promotion2":test["Promotion2"].mean(), 
    "Promotion3":test["Promotion3"].mean(), "Promotion4":test["Promotion4"].mean(), 
    "Promotion5":test["Promotion5"].mean()
}
test = test.fillna(value=means)

## 3. Data manipulation

### a) `Date`

In [4]:
import datetime as dt

def date_to_week(date):
    day, month, year = map(int, date.split('/'))
    t = dt.datetime(year, month, day) - dt.datetime(2010, 2, 5)
    return t.days // 7


def date_split(date):
    return tuple(map(int, date.split('/')))


# train
#train["Week"] = train["Date"].apply(date_to_week)
#train["Day"] = train["Date"].apply(lambda x: date_split(x)[0])
train["Month"] = train["Date"].apply(lambda x: date_split(x)[1])
train["Year"] = train["Date"].apply(lambda x: date_split(x)[2])


# test
#test["Week"] = test["Date"].apply(date_to_week)
#test["Day"] = test["Date"].apply(lambda x: date_split(x)[0])
test["Month"] = test["Date"].apply(lambda x: date_split(x)[1])
test["Year"] = test["Date"].apply(lambda x: date_split(x)[2])

### b) `IsHoliday`

In [5]:
# trian
train["IsHoliday"] = train["IsHoliday"].apply(int)

# test
test["IsHoliday"] = test["IsHoliday"].apply(int)

### c) `Store`

In [6]:
# train
train = pd.get_dummies(data=train, columns=["Store"])

# test
test = pd.get_dummies(data=test, columns=["Store"])

### d) `Promotion1`, ... , `Promotion5`

In [7]:
# Scaling
from sklearn.preprocessing import QuantileTransformer


promos = ['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']

# train
scaler = QuantileTransformer()
scaler.fit(train[promos])
train[promos] = scaler.transform(train[promos])


# test
scaler = QuantileTransformer(n_quantiles = 180)
scaler.fit(test[promos])
test[promos] = scaler.transform(test[promos])

### e) `Weekly_Sales` 

In [8]:
import numpy as np

train["Weekly_Sales"] = np.log1p(train["Weekly_Sales"])

## 4. Add new features

In [9]:
social_features = ["Temperature", "Fuel_Price", "Unemployment"]
dummy = train[social_features].copy()

## 5. Remove not using features

In [10]:
train = train.drop(columns=['id', 'Date', 'IsHoliday'])
#train = train.drop(columns=social_features)

test = test.drop(columns=['id', 'Date', 'IsHoliday'])
#test = test.drop(columns=social_features)

In [11]:
train

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,Weekly_Sales,Month,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,42.31,2.572,0.000000,0.375876,0.349850,0.000000,0.000000,8.106,14.312455,2,...,0,0,0,0,0,0,0,0,0,0
1,38.51,2.548,0.000000,0.375876,0.349850,0.000000,0.000000,8.106,14.311400,2,...,0,0,0,0,0,0,0,0,0,0
2,39.93,2.514,0.000000,0.375876,0.349850,0.000000,0.000000,8.106,14.292967,2,...,0,0,0,0,0,0,0,0,0,0
3,46.63,2.561,0.000000,0.375876,0.349850,0.000000,0.000000,8.106,14.158908,2,...,0,0,0,0,0,0,0,0,0,0
4,46.50,2.625,0.000000,0.375876,0.349850,0.000000,0.000000,8.106,14.256862,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,75.09,3.867,0.984969,0.768839,0.917918,0.967133,0.869701,8.684,13.506671,8,...,0,0,0,0,0,0,0,0,0,1
6251,75.70,3.911,0.938933,0.782810,0.890863,0.873041,0.765066,8.684,13.549608,9,...,0,0,0,0,0,0,0,0,0,1
6252,67.87,3.948,0.942905,0.375876,0.770093,0.922494,0.910518,8.684,13.462029,9,...,0,0,0,0,0,0,0,0,0,1
6253,65.32,4.038,0.902804,0.839858,0.899113,0.890731,0.969742,8.684,13.491285,9,...,0,0,0,0,0,0,0,0,0,1


In [12]:
test

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,Month,Year,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,68.55,3.617,0.921788,0.516760,0.592179,0.921788,0.648045,6.573,10,2012,...,0,0,0,0,0,0,0,0,0,0
1,62.99,3.601,0.424581,0.516760,0.396648,0.368715,0.815642,6.573,10,2012,...,0,0,0,0,0,0,0,0,0,0
2,67.97,3.594,0.245810,0.516760,0.240223,0.061453,0.446927,6.573,10,2012,...,0,0,0,0,0,0,0,0,0,0
3,69.16,3.506,0.519553,0.100559,0.284916,0.508380,0.245810,6.573,10,2012,...,0,0,0,0,0,0,0,0,0,0
4,70.27,3.617,0.821229,0.516760,0.458101,0.905028,0.703911,6.170,10,2012,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,46.97,3.755,0.050279,0.027933,0.027933,0.656425,0.039106,5.217,10,2012,...,0,0,0,0,0,0,0,0,1,0
176,64.89,3.985,0.754190,0.516760,0.597765,0.865922,0.452514,8.667,10,2012,...,0,0,0,0,0,0,0,0,0,1
177,54.47,4.000,0.396648,0.516760,0.379888,0.363128,0.715084,8.667,10,2012,...,0,0,0,0,0,0,0,0,0,1
178,56.47,3.969,0.407821,0.516760,0.156425,0.240223,0.312849,8.667,10,2012,...,0,0,0,0,0,0,0,0,0,1


----
# II. Modeling

## 1. Divide `train.csv` into training data and for predicting data

In [13]:
x_train = train.drop(columns=["Weekly_Sales"])
y_train = train["Weekly_Sales"]

## 2. Choose a suitable model

In [14]:
predictions = {}

### a) XGBoost

In [15]:
import time
from xgboost import XGBRegressor

start_t = time.time()
model = XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth = 4, n_estimators = 1000)
model.fit(x_train, y_train)
prediction = model.predict(test)
prediction = np.expm1(prediction)
predictions["XGBoost"] = prediction

print(f"[XGBoost] => time: {round(time.time() - start_t, 2)}(sec)")
print(f"{prediction[0]}, {prediction[1]}, ... , {prediction[-1]}")

[XGBoost] => time: 2.04(sec)
1554747.75, 1544342.875, ... , 741629.4375


----
# III. Submission

In [16]:
import os

def name(integer):
    return str(integer).zfill(2)
    
savetime = dt.datetime.now()
folder = "-".join(map(name, [savetime.year, savetime.month, savetime.day]))
sub_folder = name(savetime.hour) + '：' + name(savetime.minute) + '：' + name(savetime.second)

for model in predictions:
    submission["Weekly_Sales"] = predictions[model]
    os.makedirs(f"dataset/submissions/{folder}/{sub_folder}", exist_ok=True)
    submission.to_csv(f"dataset/submissions/{folder}/{sub_folder}/{model}.csv", index=False)